<a href="https://colab.research.google.com/github/RealThanosP/pred-main-mod/blob/main/Model_training_v0.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Clone repository from github

Τα πρώτα ειναι copy paste απο τον κώδικα του θανου

In [1]:
# Load the data on the server session
!git clone https://github.com/RealThanosP/pred-main-mod

# Change your working directory inside the repository
%cd pred-main-mod

Cloning into 'pred-main-mod'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 144 (delta 78), reused 2 (delta 0), pack-reused 28 (from 2)
Receiving objects: 100% (144/144), 95.49 MiB | 3.64 MiB/s, done.
Resolving deltas: 100% (78/78), done.
Updating files: 100% (29/29), done.
/content/pred-main-mod


# Review dataset

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
import os
import seaborn as sns

###Getting the sensor files in a list

In [3]:
folder_path = "data/condition+monitoring+of+hydraulic+systems"

# Define the list of sensors and their corresponding column names
sensors = [
    "PS1", "PS2", "PS3", "PS4", "PS5", "PS6",  # Pressure sensors
    "EPS1",  # Motor power
    "FS1", "FS2",  # Volume flow
    "TS1", "TS2", "TS3", "TS4",  # Temperature sensors
    "VS1",  # Vibration
    "CE",  # Cooling efficiency
    "CP",  # Cooling power
    "SE"  # Efficiency factor
]

In [4]:
# Gets all the text files from dataset
txt_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

# Keeps only the data that have a sensor name in the filename
sensor_file_path_list = [f"{folder_path}/{f}" for f in txt_files if any(sensor in f for sensor in sensors)]
sensor_file_path_list.sort()

###Loading Data

Based on the description the classification for the data is:

The target condition values are cycle-wise annotated in �profile.txt� (tab-delimited). As before, the row number represents the cycle number. The columns are

1: Cooler condition / %: 3: close to total failure 20: reduced effifiency 100: full efficiency

2: Valve condition / %: 100: optimal switching behavior 90: small lag 80: severe lag 73: close to total failure

3: Internal pump leakage: 0: no leakage 1: weak leakage 2: severe leakage

4: Hydraulic accumulator / bar: 130: optimal pressure 115: slightly reduced pressure 100: severely reduced pressure 90: close to total failure

5: stable flag: 0: conditions were stable 1: static conditions might not have been reached yet

In [5]:
# profile
profile = pd.read_csv("data/condition+monitoring+of+hydraulic+systems/profile.txt", sep="\t", header=None)
profile.columns = ["cooler_condition", "valve_condition", "internal_pump_leakage", "hydraulic_accumulator", "stable_flag"]
profile

,cooler_condition,valve_condition,internal_pump_leakage,hydraulic_accumulator,stable_flag
0,3,100,0,130,1
1,3,100,0,130,1
2,3,100,0,130,1
3,3,100,0,130,1
4,3,100,0,130,1
...,...,...,...,...,...
2200,100,100,0,90,0
2201,100,100,0,90,0
2202,100,100,0,90,0
2203,100,100,0,90,0


#Data preperation

Θα δοκιμασω να κανω ολα τα αρχεια των αισθητηρων να περιεχουν πληροφορειες mean, rms, max, min κλπ, να τα κάνω ένα ενιαίο dataset (και το profile μεσα) και να βαλω ως target column το stable flag.

In [6]:
# load all the files as seperate DataFrames with the file name as name
sensor_dfs = {}
for file_path in sensor_file_path_list:
    # remove the .txt
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    # read the file and save it with the right name
    sensor_dfs[file_name] = pd.read_csv(file_path, sep="\t", header=None)

### Feature engineering (statistical)

In [7]:
# function that does the feature engineering
def create_stats(sensor_name, df):
    # We get the DataFrame dynamically from the name
    # Calculate statistics
    stats = pd.DataFrame({
        'mean'+ sensor_name: df.mean(axis=1),
        'std'+ sensor_name: df.std(axis=1),
        'min'+ sensor_name: df.min(axis=1),
        'max'+ sensor_name: df.max(axis=1),
        'range'+ sensor_name: df.max(axis=1) - df.min(axis=1),
        'rms'+ sensor_name: (df.pow(2).mean(axis=1)).pow(0.5)
    })
    stat_names = [f'mean{sensor_name}', f'std{sensor_name}', f'min{sensor_name}', f'max{sensor_name}', f'range{sensor_name}', f'rms{sensor_name}']
    return stats

In [8]:
# dictionary to save the names as keys and the dataframes as values
stats_dfs = {}

for sensor in sensors:
    stats_dfs[f'{sensor}_stats'] = create_stats(sensor, sensor_dfs[sensor])

# Converts the keys of the dictionary to variables
globals().update(stats_dfs)

In [9]:
PS1_stats

,meanPS1,stdPS1,minPS1,maxPS1,rangePS1,rmsPS1
0,160.673492,13.939309,145.83,191.51,45.68,161.276914
1,160.603320,14.118967,145.73,191.47,45.74,161.222636
2,160.347720,14.192619,145.37,191.41,46.04,160.974495
3,160.188088,14.227803,145.14,191.34,46.20,160.818594
4,160.000472,14.276434,144.95,191.41,46.46,160.636028
...,...,...,...,...,...,...
2200,161.227572,13.987129,146.69,191.81,45.12,161.833053
2201,161.206070,14.001418,146.64,191.81,45.17,161.812867
2202,161.192120,13.988350,146.63,191.84,45.21,161.797840
2203,161.208917,13.985248,146.65,191.77,45.12,161.814305


###Feature Engineering (frequency)

In [10]:
from scipy.fft import rfft, rfftfreq

def fft_analysis(data, sampling_rate_hz):
    """
    Perform FFT analysis on each row of raw sensor data.

    Parameters:
    - data (np.ndarray or pd.DataFrame): shape (2205, N), raw time-domain signals.
    - sampling_rate_hz (int): Sampling rate in Hz.

    Returns:
    - pd.DataFrame: DataFrame of FFT magnitudes with frequency bin labels as columns.
    """
    if isinstance(data, pd.DataFrame):
        data = data.values

    num_instances, num_samples = data.shape

    # Frequency bins: same for all rows
    freqs = rfftfreq(num_samples, d=1/sampling_rate_hz)

    # Apply FFT row-wise, get magnitude spectrum
    fft_magnitudes = np.abs(rfft(data, axis=1))

    # Convert to DataFrame with frequency labels
    fft_df = pd.DataFrame(fft_magnitudes, columns=[f"{round(f, 2)}Hz" for f in freqs])

    return fft_df

In [11]:
# Plotting fft analysis
def plot_fft_spectrum(fft_df, instance_indices=[0], max_freq=None):
    """
    Plot FFT spectrum for selected instance(s).

    Parameters:
    - fft_df (pd.DataFrame): DataFrame of FFT magnitudes (rows = instances, columns = frequency bins).
    - instance_indices (list): which instance(s) to plot from the dataset.
    - max_freq (float): maximum frequency to show (e.g., 200 Hz).
    """
    # Extract frequency values from column names
    freqs = [float(col.replace('freq_', '').replace('Hz', '')) for col in fft_df.columns]

    plt.figure(figsize=(12, 6))

    for idx in instance_indices:
        plt.plot(freqs, fft_df.iloc[idx], label=f'Instance {idx}')

    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Magnitude")
    plt.title("FFT Spectrum")
    if max_freq:
        plt.xlim(0, max_freq)
    plt.grid(True)
    plt.tight_layout()
    plt.show()


Applying fft analysis on to all the dataframes

In [12]:
# Creating a dictionary with keys the name and as values a tuple (df, sampling_rate)

sampling_rates_dict = {
    'PS1': 100, 'PS2': 100, 'PS3': 100, 'PS4': 100, 'PS5': 100, 'PS6': 100,
    'EPS1': 100, 'FS1': 10, 'FS2': 10,
    'TS1': 1, 'TS2': 1, 'TS3': 1, 'TS4': 1,
    'VS1': 1,
    'CE': 1, 'CP': 1, 'SE': 1
}

#
fft_dfs = {}

for sensor in sensors:
    fft_dfs[sensor] = (fft_analysis(sensor_dfs[sensor], sampling_rates_dict[sensor]), sampling_rates_dict[sensor])

### Compine the data by category

In [21]:
# Have all the data of the same sensor-typ in different lists
ps_stats = [PS1_stats, PS2_stats, PS3_stats, PS4_stats, PS5_stats, PS6_stats]
fs_stats = [FS1_stats, FS2_stats]
ts_stats = [TS1_stats, TS2_stats, TS3_stats, TS4_stats]

In [42]:
stat_names = ["mean", "std", "min", "max", "range", "rms"]

In [115]:
def combine_sensor_stats(df_list: list[pd.DataFrame], prefix: str, stat_names: list[str]) -> pd.DataFrame:
    """
    Adds corresponding statistical features (column-wise and row-wise) from multiple DataFrames.

    Parameters:
    - df_list (list[pd.DataFrame]): List of pandas DataFrames with the same shape, columns, and index.
    - prefix (str): Prefix to apply to each statistical feature (e.g., 'PS').
    - stat_names (list[str]): List of statistical feature names (e.g., ['mean', 'std', 'min', 'max', 'range', 'rms']).

    Returns:
    - pd.DataFrame: A new DataFrame with summed columns for each statistical feature, with prefixed column names.
    """
    # Initialize individual stat containers
    mean_df, std_df, min_df, max_df, range_df, rms_df = [pd.DataFrame() for _ in range(6)]

    for df in df_list:
        # Rename columns according to the prefix
        df.columns = [f"{stat}{prefix}" for stat in stat_names]

        # Extract individual columns
        mean = df[f"mean{prefix}"]
        std = df[f"std{prefix}"]
        min_ = df[f"min{prefix}"]
        max_ = df[f"max{prefix}"]
        range_stat = df[f"range{prefix}"]
        rms = df[f"rms{prefix}"]

        # Accumulate sums
        mean_df = pd.concat([mean_df, mean], axis=1).sum(axis=1)
        std_df = pd.concat([std_df, std], axis=1).sum(axis=1)
        min_df = pd.concat([min_df, min_], axis=1).sum(axis=1)
        max_df = pd.concat([max_df, max_], axis=1).sum(axis=1)
        range_df = pd.concat([range_df, range_stat], axis=1).sum(axis=1)
        rms_df = pd.concat([rms_df, rms], axis=1).sum(axis=1)

    # Combine all the summed stats into a final DataFrame
    summed_df = pd.concat([mean_df, std_df, min_df, max_df, range_df, rms_df], axis=1)
    summed_df.columns = [f"{stat}{prefix}" for stat in stat_names]

    return summed_df

In [107]:
combined_stats_ps = combine_sensor_stats(ps_stats, "PS", stat_names)
combined_stats_fs = combine_sensor_stats(fs_stats, "FS", stat_names)
combined_stats_ts = combine_sensor_stats(ts_stats, "TS", stat_names)

###Compine all the data

In [117]:
combined_data = pd.concat([combined_stats_ps, combined_stats_fs, combined_stats_ts, CE_stats, CP_stats, EPS1_stats, VS1_stats, SE_stats], axis=1)
combined_data

,meanPS,stdPS,minPS,maxPS,rangePS,rmsPS,meanFS,stdFS,minFS,maxFS,...,minVS1,maxVS1,rangeVS1,rmsVS1,meanSE,stdSE,minSE,maxSE,rangeSE,rmsSE
0,291.702147,62.173041,165.079,378.442,213.363,302.226405,17.014407,3.107199,10.148,29.156,...,0.532,0.624,0.092,0.577575,59.157183,23.763984,0.0,79.568,79.568,63.677996
1,291.099075,62.181253,164.741,378.404,213.663,301.620368,17.118413,3.025410,10.352,29.165,...,0.524,0.626,0.102,0.566494,59.335617,23.857918,0.0,80.441,80.441,63.878236
2,290.437285,62.200027,164.173,378.470,214.297,300.961880,17.084772,3.040609,10.296,29.141,...,0.529,0.662,0.133,0.577683,59.543150,23.923381,0.0,80.824,80.824,64.095055
3,289.975727,62.210002,163.760,377.043,213.283,300.502962,17.023243,3.046925,10.241,29.278,...,0.527,0.645,0.118,0.570233,59.794900,24.023005,0.0,80.930,80.930,64.365491
4,289.474011,62.140087,163.435,379.427,215.992,299.978761,16.928058,3.040721,10.179,29.188,...,0.524,0.660,0.136,0.578321,59.455267,23.972262,0.0,81.100,81.100,64.031400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,303.033461,62.239275,176.520,398.942,222.422,313.581549,16.874445,3.015213,10.147,30.452,...,0.512,0.618,0.106,0.551627,59.033100,23.635352,0.0,77.752,77.752,63.515559
2201,303.001287,62.247484,176.445,398.641,222.196,313.547839,16.869948,3.020499,10.133,30.541,...,0.507,0.584,0.077,0.547897,59.068000,23.638396,0.0,77.592,77.592,63.549111
2202,302.945512,62.230947,176.421,398.842,222.421,313.492210,16.869448,3.006082,10.127,30.296,...,0.502,0.597,0.095,0.545755,59.132350,23.633405,0.0,77.773,77.773,63.607104
2203,303.023387,62.230756,176.431,398.938,222.507,313.565487,16.862482,3.018602,10.129,30.510,...,0.502,0.585,0.083,0.537302,58.970800,23.600817,0.0,77.568,77.568,63.445019


In [114]:
# the dataset for scaling
profile_without_first_and_last_columns = profile.iloc[:, :-1]
data_for_scalling = pd.concat([combined_data, profile_without_first_and_last_columns], axis=1)
data_for_scalling

,meanFS,stdFS,minFS,maxFS,rangeFS,rmsFS,meanPS,stdPS,minPS,maxPS,...,meanSE,stdSE,minSE,maxSE,rangeSE,rmsSE,cooler_condition,valve_condition,internal_pump_leakage,hydraulic_accumulator
0,17.014407,3.107199,10.148,29.156,19.008,17.660220,291.702147,62.173041,165.079,378.442,...,59.157183,23.763984,0.0,79.568,79.568,63.677996,3,100,0,130
1,17.118413,3.025410,10.352,29.165,18.813,17.759604,291.099075,62.181253,164.741,378.404,...,59.335617,23.857918,0.0,80.441,80.441,63.878236,3,100,0,130
2,17.084772,3.040609,10.296,29.141,18.845,17.728965,290.437285,62.200027,164.173,378.470,...,59.543150,23.923381,0.0,80.824,80.824,64.095055,3,100,0,130
3,17.023243,3.046925,10.241,29.278,19.037,17.669683,289.975727,62.210002,163.760,377.043,...,59.794900,24.023005,0.0,80.930,80.930,64.365491,3,100,0,130
4,16.928058,3.040721,10.179,29.188,19.009,17.573631,289.474011,62.140087,163.435,379.427,...,59.455267,23.972262,0.0,81.100,81.100,64.031400,3,100,0,130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,16.874445,3.015213,10.147,30.452,20.305,17.515910,303.033461,62.239275,176.520,398.942,...,59.033100,23.635352,0.0,77.752,77.752,63.515559,100,100,0,90
2201,16.869948,3.020499,10.133,30.541,20.408,17.512984,303.001287,62.247484,176.445,398.641,...,59.068000,23.638396,0.0,77.592,77.592,63.549111,100,100,0,90
2202,16.869448,3.006082,10.127,30.296,20.169,17.506470,302.945512,62.230947,176.421,398.842,...,59.132350,23.633405,0.0,77.773,77.773,63.607104,100,100,0,90
2203,16.862482,3.018602,10.129,30.510,20.381,17.505467,303.023387,62.230756,176.431,398.938,...,58.970800,23.600817,0.0,77.568,77.568,63.445019,100,100,0,90


### Scaling the data

εδω καλο θα ηταν να ονοματησω σωστα τισ στηλες του Dataframe
**FIX το εκανε ο Θάνος :) **

In [118]:
from sklearn.preprocessing import MinMaxScaler
scaler = StandardScaler()

scaled_data = scaler.fit_transform(data_for_scalling)
scaled_data_dtset = pd.DataFrame(scaled_data, columns=data_for_scalling.columns) # Just a columns argument needed
scaled_data_dtset

,meanFS,stdFS,minFS,maxFS,rangeFS,rmsFS,meanPS,stdPS,minPS,maxPS,...,meanSE,stdSE,minSE,maxSE,rangeSE,rmsSE,cooler_condition,valve_condition,internal_pump_leakage,hydraulic_accumulator
0,0.875728,0.261479,1.206382,-1.516235,-2.694839,0.999008,-0.070639,-0.360957,0.144353,-1.897972,...,0.431929,-0.202694,0.0,-0.063981,-0.063981,0.456140,-0.902469,0.87141,-0.819277,1.387554
1,0.953815,-0.166577,1.653169,-1.496113,-3.122420,1.091810,-0.126748,-0.359408,0.096028,-1.904578,...,0.451847,-0.153763,0.0,0.091880,0.091880,0.485438,-0.902469,0.87141,-0.819277,1.387554
2,0.928557,-0.087029,1.530521,-1.549772,-3.052253,1.063200,-0.188319,-0.355868,0.014818,-1.893104,...,0.475014,-0.119662,0.0,0.160258,0.160258,0.517161,-0.902469,0.87141,-0.819277,1.387554
3,0.882362,-0.053975,1.410064,-1.243469,-2.631250,1.007845,-0.231262,-0.353987,-0.044231,-2.141165,...,0.503117,-0.067767,0.0,0.179183,0.179183,0.556728,-0.902469,0.87141,-0.819277,1.387554
4,0.810898,-0.086447,1.274276,-1.444690,-2.692646,0.918154,-0.277941,-0.367171,-0.090698,-1.726746,...,0.465204,-0.094199,0.0,0.209534,0.209534,0.507847,-0.902469,0.87141,-0.819277,1.387554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,0.770646,-0.219946,1.204192,1.381345,0.149123,0.864256,0.983604,-0.348467,1.780136,1.665621,...,0.418077,-0.269700,0.0,-0.388199,-0.388199,0.432374,1.386695,0.87141,-0.819277,-1.046703
2201,0.767270,-0.192279,1.173530,1.580330,0.374974,0.861524,0.980611,-0.346919,1.769412,1.613297,...,0.421973,-0.268115,0.0,-0.416764,-0.416764,0.437283,1.386695,0.87141,-0.819277,-1.046703
2202,0.766894,-0.267734,1.160389,1.032563,-0.149087,0.855441,0.975422,-0.350037,1.765981,1.648237,...,0.429156,-0.270715,0.0,-0.384450,-0.384450,0.445768,1.386695,0.87141,-0.819277,-1.046703
2203,0.761664,-0.202208,1.164769,1.511021,0.315770,0.854505,0.982667,-0.350073,1.767411,1.664925,...,0.411123,-0.287690,0.0,-0.421049,-0.421049,0.422054,1.386695,0.87141,-0.819277,-1.046703


In [119]:
profile_last_column = profile.iloc[:, -1]
total_data = pd.concat([scaled_data_dtset, profile_last_column], axis = 1)
total_data

,meanFS,stdFS,minFS,maxFS,rangeFS,rmsFS,meanPS,stdPS,minPS,maxPS,...,stdSE,minSE,maxSE,rangeSE,rmsSE,cooler_condition,valve_condition,internal_pump_leakage,hydraulic_accumulator,stable_flag
0,0.875728,0.261479,1.206382,-1.516235,-2.694839,0.999008,-0.070639,-0.360957,0.144353,-1.897972,...,-0.202694,0.0,-0.063981,-0.063981,0.456140,-0.902469,0.87141,-0.819277,1.387554,1
1,0.953815,-0.166577,1.653169,-1.496113,-3.122420,1.091810,-0.126748,-0.359408,0.096028,-1.904578,...,-0.153763,0.0,0.091880,0.091880,0.485438,-0.902469,0.87141,-0.819277,1.387554,1
2,0.928557,-0.087029,1.530521,-1.549772,-3.052253,1.063200,-0.188319,-0.355868,0.014818,-1.893104,...,-0.119662,0.0,0.160258,0.160258,0.517161,-0.902469,0.87141,-0.819277,1.387554,1
3,0.882362,-0.053975,1.410064,-1.243469,-2.631250,1.007845,-0.231262,-0.353987,-0.044231,-2.141165,...,-0.067767,0.0,0.179183,0.179183,0.556728,-0.902469,0.87141,-0.819277,1.387554,1
4,0.810898,-0.086447,1.274276,-1.444690,-2.692646,0.918154,-0.277941,-0.367171,-0.090698,-1.726746,...,-0.094199,0.0,0.209534,0.209534,0.507847,-0.902469,0.87141,-0.819277,1.387554,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,0.770646,-0.219946,1.204192,1.381345,0.149123,0.864256,0.983604,-0.348467,1.780136,1.665621,...,-0.269700,0.0,-0.388199,-0.388199,0.432374,1.386695,0.87141,-0.819277,-1.046703,0
2201,0.767270,-0.192279,1.173530,1.580330,0.374974,0.861524,0.980611,-0.346919,1.769412,1.613297,...,-0.268115,0.0,-0.416764,-0.416764,0.437283,1.386695,0.87141,-0.819277,-1.046703,0
2202,0.766894,-0.267734,1.160389,1.032563,-0.149087,0.855441,0.975422,-0.350037,1.765981,1.648237,...,-0.270715,0.0,-0.384450,-0.384450,0.445768,1.386695,0.87141,-0.819277,-1.046703,0
2203,0.761664,-0.202208,1.164769,1.511021,0.315770,0.854505,0.982667,-0.350073,1.767411,1.664925,...,-0.287690,0.0,-0.421049,-0.421049,0.422054,1.386695,0.87141,-0.819277,-1.046703,0


# Feature creation

### Reviewing the data

In [ ]:
total_data

,meanFS1,stdFS1,minFS1,maxFS1,rangeFS1,rmsFS1,meanFS2,stdFS2,minFS2,maxFS2,...,minSE,maxSE,rangeSE,rmsSE,cooler_condition,valve_condition,internal_pump_leakage,hydraulic_accumulator,stable_flag,maintenance_score
0,0.495101,-0.130443,-0.021301,-3.143240,-3.143235,0.546497,1.458639,18.562454,1.206381,1.677334,...,0.0,-0.063981,-0.063981,0.456140,-0.902469,0.87141,-0.819277,1.387554,1,0.360636
1,0.500427,-0.179022,-0.021301,-3.138813,-3.138809,0.548169,1.677960,0.497722,1.653167,1.693157,...,0.0,0.091880,0.091880,0.485438,-0.902469,0.87141,-0.819277,1.387554,1,0.360636
2,0.503533,-0.136685,-0.021301,-3.169798,-3.169794,0.556338,1.595919,2.288859,1.530520,1.670553,...,0.0,0.160258,0.160258,0.517161,-0.902469,0.87141,-0.819277,1.387554,1,0.360636
3,0.505511,-0.105318,-0.021301,-2.731581,-2.731577,0.561995,1.454379,2.385506,1.410063,1.532668,...,0.0,0.179183,0.179183,0.556728,-0.902469,0.87141,-0.819277,1.387554,1,0.360636
4,0.476211,-0.150963,-0.021301,-2.775846,-2.775842,0.520934,1.309819,2.987783,1.274275,1.374439,...,0.0,0.209534,0.209534,0.507847,-0.902469,0.87141,-0.819277,1.387554,1,0.360636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,0.475845,-0.204087,-0.021301,0.216421,0.216422,0.515060,1.191293,-0.860944,1.204191,1.175523,...,0.0,-0.388199,-0.388199,0.432374,1.386695,0.87141,-0.819277,-1.046703,0,0.382405
2201,0.478025,-0.181232,-0.021301,0.393479,0.393479,0.520099,1.176268,-0.620172,1.173529,1.195866,...,0.0,-0.416764,-0.416764,0.437283,1.386695,0.87141,-0.819277,-1.046703,0,0.382405
2202,0.479086,-0.257953,-0.021301,-0.131053,-0.131052,0.513611,1.172717,-0.600236,1.160388,1.177783,...,0.0,-0.384450,-0.384450,0.445768,1.386695,0.87141,-0.819277,-1.046703,0,0.382405
2203,0.470227,-0.191012,-0.021301,0.331509,0.331509,0.509411,1.177575,-0.632400,1.164768,1.189085,...,0.0,-0.421049,-0.421049,0.422054,1.386695,0.87141,-0.819277,-1.046703,0,0.382405


### Create Coefficient of Variance (CV)
---
Measures relative variability on the sensor measurements. High CV means erratic sensor motion which may impact maintenance issues.

---
**Formula:** \begin{align*}
          \text{CV} &= \frac{\text{std}}{\text{mean}}
          \end{align*}

In [ ]:
def compute_coefficient_variance(row, sensors):
    cv_values = {}
    for name in sensors:
        mean = row[f'mean{name}']
        std = row[f'std{name}']
        cv_values[f'cv{name}'] = std / (mean + 1e-6)  # Avoid division by zero
    return cv_values

In [ ]:
# Apply to the dataframe
copy_data = total_data.copy()
cv_df = copy_data.apply(lambda row: compute_coefficient_variance(row, sensors), axis=1, result_type='expand')

# Scale the data
cv_df = pd.DataFrame(scaler.fit_transform(cv_df), columns=cv_df.columns)

# Get the insertion point: 6 columns before the end
insert_at = total_data.shape[1] - profile.columns.size

# Split original DataFrame into two parts
df_first = total_data.iloc[:, :insert_at]
df_last = total_data.iloc[:, insert_at:]

# places the cv columns before the profile columns and after the statistical data
copy_data = pd.concat([df_first, cv_df, df_last], axis=1)
copy_data

,meanFS1,stdFS1,minFS1,maxFS1,rangeFS1,rmsFS1,meanFS2,stdFS2,minFS2,maxFS2,...,cvTS4,cvVS1,cvCE,cvCP,cvSE,valve_condition,internal_pump_leakage,hydraulic_accumulator,stable_flag,maintenance_score
0,0.495101,-0.130443,-0.021301,-3.143240,-3.143235,0.546497,1.458639,18.562454,1.206381,1.677334,...,-3.191367,0.046869,22.076806,33.832675,0.062632,0.87141,-0.819277,1.387554,1,0.360636
1,0.500427,-0.179022,-0.021301,-3.138813,-3.138809,0.548169,1.677960,0.497722,1.653167,1.693157,...,-1.673065,0.041463,-7.677607,-0.636859,0.067658,0.87141,-0.819277,1.387554,1,0.360636
2,0.503533,-0.136685,-0.021301,-3.169798,-3.169794,0.556338,1.595919,2.288859,1.530520,1.670553,...,-1.308722,0.033209,-1.189884,0.029237,0.071103,0.87141,-0.819277,1.387554,1,0.360636
3,0.505511,-0.105318,-0.021301,-2.731581,-2.731577,0.561995,1.454379,2.385506,1.410063,1.532668,...,-1.401823,0.035560,-0.492636,0.016621,0.075671,0.87141,-0.819277,1.387554,1,0.360636
4,0.476211,-0.150963,-0.021301,-2.775846,-2.775842,0.520934,1.309819,2.987783,1.274275,1.374439,...,-1.579120,0.038024,-0.031357,0.046900,0.073029,0.87141,-0.819277,1.387554,1,0.360636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,0.475845,-0.204087,-0.021301,0.216421,0.216422,0.515060,1.191293,-0.860944,1.204191,1.175523,...,0.040985,0.035276,-0.121441,-0.056749,0.055780,0.87141,-0.819277,-1.046703,0,0.382405
2201,0.478025,-0.181232,-0.021301,0.393479,0.393479,0.520099,1.176268,-0.620172,1.173529,1.195866,...,0.025909,0.041215,0.183023,0.049303,0.056158,0.87141,-0.819277,-1.046703,0,0.382405
2202,0.479086,-0.257953,-0.021301,-0.131053,-0.131052,0.513611,1.172717,-0.600236,1.160388,1.177783,...,0.006500,0.038723,0.324309,0.131072,0.056337,0.87141,-0.819277,-1.046703,0,0.382405
2203,0.470227,-0.191012,-0.021301,0.331509,0.331509,0.509411,1.177575,-0.632400,1.164768,1.189085,...,0.023894,0.041473,-0.113212,-0.068310,0.053649,0.87141,-0.819277,-1.046703,0,0.382405


### Create Signal Stability Ratio
---
Low stability ratio indicates fluctuations and lead to machine failure. This feature is comparing how close is the machine to its stability limits. Meaning that it compares the mean of the sensor's values to it's range

---

\begin{align*}
          \text{stability} &= \frac{\text{mean}}{\text{range}+ 1 \times 10^{-6}}
          \end{align*}



In [ ]:
def compute_stability(row, sensors):
    stability_values = {}
    for name in sensors:
        mean = row[f'mean{name}']
        range = row[f'range{name}']
        stability_values[f'stability{name}'] = mean / (range + 10**(-6))
    return stability_values

stability_df = copy_data.apply(lambda row: compute_stability(row, sensors), axis=1, result_type='expand')
# Scale the data
stability_df = pd.DataFrame(scaler.fit_transform(stability_df), columns=stability_df.columns)

# Get the insertion point: 6 columns before the end
insert_at = total_data.shape[1] - 6

# Split original DataFrame into two parts
df_first = total_data.iloc[:, :insert_at]
df_last = total_data.iloc[:, insert_at:]

# places the cv columns before the profile columns and after the statistical data
copy_data = pd.concat([df_first, stability_df, df_last], axis=1)
copy_data

,meanFS1,stdFS1,minFS1,maxFS1,rangeFS1,rmsFS1,meanFS2,stdFS2,minFS2,maxFS2,...,stabilityVS1,stabilityCE,stabilityCP,stabilitySE,cooler_condition,valve_condition,internal_pump_leakage,hydraulic_accumulator,stable_flag,maintenance_score
0,0.495101,-0.130443,-0.021301,-3.143240,-3.143235,0.546497,1.458639,18.562454,1.206381,1.677334,...,-0.052766,-0.012037,0.019523,-0.011559,-0.902469,0.87141,-0.819277,1.387554,1,0.360636
1,0.500427,-0.179022,-0.021301,-3.138813,-3.138809,0.548169,1.677960,0.497722,1.653167,1.693157,...,-0.050826,-0.013201,-0.003400,0.071536,-0.902469,0.87141,-0.819277,1.387554,1,0.360636
2,0.503533,-0.136685,-0.021301,-3.169798,-3.169794,0.556338,1.595919,2.288859,1.530520,1.670553,...,-0.048344,-0.017628,1.399002,0.057624,-0.902469,0.87141,-0.819277,1.387554,1,0.360636
3,0.505511,-0.105318,-0.021301,-2.731581,-2.731577,0.561995,1.454379,2.385506,1.410063,1.532668,...,-0.049578,-0.022119,3.268475,0.056511,-0.902469,0.87141,-0.819277,1.387554,1,0.360636
4,0.476211,-0.150963,-0.021301,-2.775846,-2.775842,0.520934,1.309819,2.987783,1.274275,1.374439,...,-0.047814,-0.047884,-1.600777,0.052326,-0.902469,0.87141,-0.819277,1.387554,1,0.360636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,0.475845,-0.204087,-0.021301,0.216421,0.216422,0.515060,1.191293,-0.860944,1.204191,1.175523,...,-0.048499,-0.057282,-0.126423,0.028847,1.386695,0.87141,-0.819277,-1.046703,0,0.382405
2201,0.478025,-0.181232,-0.021301,0.393479,0.393479,0.520099,1.176268,-0.620172,1.173529,1.195866,...,-0.050745,0.005148,0.148505,0.029306,1.386695,0.87141,-0.819277,-1.046703,0,0.382405
2202,0.479086,-0.257953,-0.021301,-0.131053,-0.131052,0.513611,1.172717,-0.600236,1.160388,1.177783,...,-0.049073,0.013008,1.070343,0.028567,1.386695,0.87141,-0.819277,-1.046703,0,0.382405
2203,0.470227,-0.191012,-0.021301,0.331509,0.331509,0.509411,1.177575,-0.632400,1.164768,1.189085,...,-0.049331,0.168655,-0.418248,0.029563,1.386695,0.87141,-0.819277,-1.046703,0,0.382405


### Creating Exponential Moving Average (EMA)
---
Smoothens the signal and helps detect long-term trends on slow mechanical tear

In [ ]:
# Code here

### Creating Frequency Domain Features (FFT)
---
Frequency peaks reveal unwanted mechanical wear.
---
-    Band Power Features
    - Average/sum of FFT magnitudes in frequency bands

-    Top-N Peak Features

    - Frequencies and magnitudes of top N peaks
    - Useful for detecting dominant fault frequencies

-     Spectral Descriptors

        - Spectral Centroid
        - Spectral Entropy
        - Spectral Flatness

Band power feature
Average sum of FFT magnitudes in frequency bands

In [ ]:
def band_power(fft_df, freqs, band_limits, name):
    """
    Calculate band power from FFT magnitudes.

    Args:
        fft_df (pd.DataFrame): The DataFrame containing FFT magnitudes.
        freqs (np.ndarray): Array of frequencies.
        band_limits (list): List of frequency band limits.
        name (str): Name of the sensor

    Returns:
        pd.DataFrame: DataFrame with band power features.
    """
    band_features = {}
    for low, high in band_limits:
        band_name = f"{name}{int(low)}_{int(high)}Hz"
        mask = (freqs >= low) & (freqs < high)
        band_features[band_name] = fft_df.iloc[:, mask].mean(axis=1)

    band_df = pd.DataFrame(band_features)
    band_df = band_df.dropna(axis=1, how='any')

    return band_df

In [ ]:
# Helper funcion to create correctly the freqs array
def get_freqs(fft_df, sampling_rate_hz):
    n_bins = fft_df.shape[1]  # number of FFT bins (columns)
    freqs = np.fft.rfftfreq(n=2 * (n_bins - 1), d=1 / sampling_rate_hz)
    return freqs


Top-N-peaks (found that N=3 is optimal)

In [ ]:
from scipy.signal import find_peaks
import pandas as pd
import numpy as np

def top_peaks(fft_df, freqs, name, n=3):
    """
    Extracts top N frequency peaks and returns them in a structured DataFrame.

    Parameters:
    - fft_df (pd.DataFrame): FFT magnitudes per instance (rows) x frequency bins (columns)
    - freqs (np.ndarray): frequency bin centers corresponding to FFT columns
    - n (int): Number of top peaks to extract
    - name (str): Name of the sensor

    Returns:
    - pd.DataFrame: with columns [peak1_freq, peak1_mag, peak2_freq, peak2_mag, ...]
    """
    peak_features = []

    for _, row in fft_df.iterrows():
        peaks, _ = find_peaks(row.values, prominence=1)
        peak_mags = row.values[peaks]
        top_indices = np.argsort(peak_mags)[-n:]

        # Sort by descending magnitude for consistency
        sorted_indices = top_indices[np.argsort(-peak_mags[top_indices])]

        peak_freqs = freqs[peaks[sorted_indices]]
        peak_vals = peak_mags[sorted_indices]

        # Pad if fewer than n peaks
        if len(peak_freqs) < n:
            pad_len = n - len(peak_freqs)
            peak_freqs = np.pad(peak_freqs, (0, pad_len), constant_values=np.nan)
            peak_vals = np.pad(peak_vals, (0, pad_len), constant_values=np.nan)

        feature_row = list(np.column_stack((peak_freqs, peak_vals)).flatten())
        peak_features.append(feature_row)

    columns = [f"{name}_peak{i+1}_freq" if j % 2 == 0 else f"{name}_peak{i+1}_mag"
               for i in range(n) for j in range(2)]

    top_peaks_df = pd.DataFrame(peak_features, columns=columns)
    top_peaks_df = top_peaks_df.dropna(axis=1, how='any')

    return top_peaks_df


Spectral Centroid, Entropy and Flatness

In [ ]:
def spectral_features(fft_df, freqs, name):
    """
    Calculate spectral centroid, entropy, and flatness for each row in the FFT dataframe.

    Returns:
        pd.DataFrame with columns: [spectral_centroid, spectral_entropy, spectral_flatness]
    """
    spectral_data = []

    for _, row in fft_df.iterrows():
        magnitudes = row.values
        mag_sum = np.sum(magnitudes)

        # Avoid divide-by-zero for silent signal rows
        if mag_sum == 0:
            centroid = np.nan
            entropy = np.nan
            flatness = np.nan
        else:
            probs = magnitudes / mag_sum

            # Spectral Centroid
            centroid = np.sum(freqs * probs)

            # Spectral Entropy
            entropy = -np.sum(probs * np.log2(probs + 1e-10))  # Add epsilon to avoid log(0)

            # Spectral Flatness
            geometric_mean = np.exp(np.mean(np.log(magnitudes + 1e-10)))
            arithmetic_mean = np.mean(magnitudes)
            flatness = geometric_mean / (arithmetic_mean + 1e-10)

        spectral_data.append([centroid, entropy, flatness])

    return pd.DataFrame(spectral_data, columns=[f"{name}_spectral_centroid", f"{name}_spectral_entropy", f"{name}_spectral_flatness"])

Concating all frequency features to the final DataFrame

In [ ]:
# Computing band powers for all the sensors
stat_freq_data = total_data.copy()
for sensor in sensors:
    # Band power
    freqs = get_freqs(fft_dfs[sensor][0], fft_dfs[sensor][1])
    band_power_df = band_power(fft_dfs[sensor][0], freqs, [(0,10),(10, 50)], name=sensor)

    # Top N peaks
    top_peaks_df = top_peaks(fft_dfs[sensor][0], freqs, name=sensor)

    # Spectral features
    spectral_df = spectral_features(fft_dfs[sensor][0], freqs, name=sensor)

    # Get the insertion point: 6 columns before the end
    insert_at = stat_freq_data.shape[1] - 5
    # Split original DataFrame into two parts
    df_first = stat_freq_data.iloc[:, :insert_at]
    df_last = stat_freq_data.iloc[:, insert_at:]

    # Concating everything in the final table
    stat_freq_data = pd.concat([df_first, band_power_df, df_last], axis=1)
    stat_freq_data = pd.concat([df_first, top_peaks_df, df_last], axis=1)
    stat_freq_data = pd.concat([df_first, spectral_df, df_last], axis=1)

# The stat_freq_data contains statistical data for each sensor
stat_freq_data

,meanFS1,stdFS1,minFS1,maxFS1,rangeFS1,rmsFS1,meanFS2,stdFS2,minFS2,maxFS2,...,CP_spectral_entropy,CP_spectral_flatness,SE_spectral_centroid,SE_spectral_entropy,SE_spectral_flatness,valve_condition,internal_pump_leakage,hydraulic_accumulator,stable_flag,maintenance_score
0,0.495101,-0.130443,-0.021301,-3.143240,-3.143235,0.546497,1.458639,18.562454,1.206381,1.677334,...,1.759044,0.186150,0.071530,3.126806,0.300456,0.87141,-0.819277,1.387554,1,0.360636
1,0.500427,-0.179022,-0.021301,-3.138813,-3.138809,0.548169,1.677960,0.497722,1.653167,1.693157,...,1.042249,0.100353,0.070487,3.108935,0.320467,0.87141,-0.819277,1.387554,1,0.360636
2,0.503533,-0.136685,-0.021301,-3.169798,-3.169794,0.556338,1.595919,2.288859,1.530520,1.670553,...,0.508577,0.042704,0.070951,3.118370,0.298354,0.87141,-0.819277,1.387554,1,0.360636
3,0.505511,-0.105318,-0.021301,-2.731581,-2.731577,0.561995,1.454379,2.385506,1.410063,1.532668,...,0.531040,0.046708,0.071274,3.123157,0.311515,0.87141,-0.819277,1.387554,1,0.360636
4,0.476211,-0.150963,-0.021301,-2.775846,-2.775842,0.520934,1.309819,2.987783,1.274275,1.374439,...,0.448211,0.036692,0.069130,3.085876,0.309750,0.87141,-0.819277,1.387554,1,0.360636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,0.475845,-0.204087,-0.021301,0.216421,0.216422,0.515060,1.191293,-0.860944,1.204191,1.175523,...,0.248128,0.021002,0.067834,3.056402,0.310444,0.87141,-0.819277,-1.046703,0,0.382405
2201,0.478025,-0.181232,-0.021301,0.393479,0.393479,0.520099,1.176268,-0.620172,1.173529,1.195866,...,0.263789,0.018977,0.067764,3.053143,0.306332,0.87141,-0.819277,-1.046703,0,0.382405
2202,0.479086,-0.257953,-0.021301,-0.131053,-0.131052,0.513611,1.172717,-0.600236,1.160388,1.177783,...,0.280006,0.022524,0.068579,3.068287,0.283098,0.87141,-0.819277,-1.046703,0,0.382405
2203,0.470227,-0.191012,-0.021301,0.331509,0.331509,0.509411,1.177575,-0.632400,1.164768,1.189085,...,0.215004,0.015761,0.067792,3.053789,0.276487,0.87141,-0.819277,-1.046703,0,0.382405


### Creating Remaining Useful Life (RUL)
---
This feature will

# Targets
---
The features that we want the model to ultimately predict

### Reviewing dataset

In [ ]:
stat_freq_data

,meanFS1,stdFS1,minFS1,maxFS1,rangeFS1,rmsFS1,meanFS2,stdFS2,minFS2,maxFS2,...,CP_spectral_entropy,CP_spectral_flatness,SE_spectral_centroid,SE_spectral_entropy,SE_spectral_flatness,valve_condition,internal_pump_leakage,hydraulic_accumulator,stable_flag,maintenance_score
0,0.495101,-0.130443,-0.021301,-3.143240,-3.143235,0.546497,1.458639,18.562454,1.206381,1.677334,...,1.759044,0.186150,0.071530,3.126806,0.300456,0.87141,-0.819277,1.387554,1,0.360636
1,0.500427,-0.179022,-0.021301,-3.138813,-3.138809,0.548169,1.677960,0.497722,1.653167,1.693157,...,1.042249,0.100353,0.070487,3.108935,0.320467,0.87141,-0.819277,1.387554,1,0.360636
2,0.503533,-0.136685,-0.021301,-3.169798,-3.169794,0.556338,1.595919,2.288859,1.530520,1.670553,...,0.508577,0.042704,0.070951,3.118370,0.298354,0.87141,-0.819277,1.387554,1,0.360636
3,0.505511,-0.105318,-0.021301,-2.731581,-2.731577,0.561995,1.454379,2.385506,1.410063,1.532668,...,0.531040,0.046708,0.071274,3.123157,0.311515,0.87141,-0.819277,1.387554,1,0.360636
4,0.476211,-0.150963,-0.021301,-2.775846,-2.775842,0.520934,1.309819,2.987783,1.274275,1.374439,...,0.448211,0.036692,0.069130,3.085876,0.309750,0.87141,-0.819277,1.387554,1,0.360636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,0.475845,-0.204087,-0.021301,0.216421,0.216422,0.515060,1.191293,-0.860944,1.204191,1.175523,...,0.248128,0.021002,0.067834,3.056402,0.310444,0.87141,-0.819277,-1.046703,0,0.382405
2201,0.478025,-0.181232,-0.021301,0.393479,0.393479,0.520099,1.176268,-0.620172,1.173529,1.195866,...,0.263789,0.018977,0.067764,3.053143,0.306332,0.87141,-0.819277,-1.046703,0,0.382405
2202,0.479086,-0.257953,-0.021301,-0.131053,-0.131052,0.513611,1.172717,-0.600236,1.160388,1.177783,...,0.280006,0.022524,0.068579,3.068287,0.283098,0.87141,-0.819277,-1.046703,0,0.382405
2203,0.470227,-0.191012,-0.021301,0.331509,0.331509,0.509411,1.177575,-0.632400,1.164768,1.189085,...,0.215004,0.015761,0.067792,3.053789,0.276487,0.87141,-0.819277,-1.046703,0,0.382405


### Creating the maintenance_score (based on the profile data)

Defining the weights

**NOTE:** Τα weights τα έβαλα ψιλο-αφθαίρετα

In [ ]:
weights = {
    'cooler_condition': 0.6,   # Heavily impacts efficiency
    'valve_condition': 0.5,    # Valve issues degrade performance quickly
    'internal_pump_leakage': 0.7,  # Leakage is a major issue
    'hydraulic_accumulator': 0.5,    # Pressure drop affects system performance
    'stable_flag': -0.3  # If unstable, slightly lower maintenance score (uncertainty)
}

Υπολογίζουμε το maintenance_score με βάση τα παραπάνω weights

In [ ]:
def compute_maintenance_score(row, weights):
    # Compute an initial score using the provided weights
    base_score = (row['cooler_condition'] * weights['cooler_condition'] +
                  row['valve_condition'] * weights['valve_condition'] +
                  row['internal_pump_leakage'] * weights['internal_pump_leakage'] +
                  row['hydraulic_accumulator'] * weights['hydraulic_accumulator'])

    # If stable_flag is 0 (machine is stable), reduce the score
    if row['stable_flag'] == 0:
        return base_score * (1 + weights['stable_flag'])  # Apply stability weight

    return base_score  # Return the raw score without capping at 1

total_data['maintenance_score_raw'] = total_data.apply(lambda row: compute_maintenance_score(row, weights), axis=1)

# Scale maintenance_score to minMax
min_max_scaler = MinMaxScaler()
total_data['maintenance_score'] = min_max_scaler.fit_transform(total_data[['maintenance_score_raw']])
total_data.drop(columns="maintenance_score_raw", inplace=True)
total_data

,meanFS1,stdFS1,minFS1,maxFS1,rangeFS1,rmsFS1,meanFS2,stdFS2,minFS2,maxFS2,...,minSE,maxSE,rangeSE,rmsSE,cooler_condition,valve_condition,internal_pump_leakage,hydraulic_accumulator,stable_flag,maintenance_score
0,0.495101,-0.130443,-0.021301,-3.143240,-3.143235,0.546497,1.458639,18.562454,1.206381,1.677334,...,0.0,-0.063981,-0.063981,0.456140,-0.902469,0.87141,-0.819277,1.387554,1,0.360636
1,0.500427,-0.179022,-0.021301,-3.138813,-3.138809,0.548169,1.677960,0.497722,1.653167,1.693157,...,0.0,0.091880,0.091880,0.485438,-0.902469,0.87141,-0.819277,1.387554,1,0.360636
2,0.503533,-0.136685,-0.021301,-3.169798,-3.169794,0.556338,1.595919,2.288859,1.530520,1.670553,...,0.0,0.160258,0.160258,0.517161,-0.902469,0.87141,-0.819277,1.387554,1,0.360636
3,0.505511,-0.105318,-0.021301,-2.731581,-2.731577,0.561995,1.454379,2.385506,1.410063,1.532668,...,0.0,0.179183,0.179183,0.556728,-0.902469,0.87141,-0.819277,1.387554,1,0.360636
4,0.476211,-0.150963,-0.021301,-2.775846,-2.775842,0.520934,1.309819,2.987783,1.274275,1.374439,...,0.0,0.209534,0.209534,0.507847,-0.902469,0.87141,-0.819277,1.387554,1,0.360636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,0.475845,-0.204087,-0.021301,0.216421,0.216422,0.515060,1.191293,-0.860944,1.204191,1.175523,...,0.0,-0.388199,-0.388199,0.432374,1.386695,0.87141,-0.819277,-1.046703,0,0.382405
2201,0.478025,-0.181232,-0.021301,0.393479,0.393479,0.520099,1.176268,-0.620172,1.173529,1.195866,...,0.0,-0.416764,-0.416764,0.437283,1.386695,0.87141,-0.819277,-1.046703,0,0.382405
2202,0.479086,-0.257953,-0.021301,-0.131053,-0.131052,0.513611,1.172717,-0.600236,1.160388,1.177783,...,0.0,-0.384450,-0.384450,0.445768,1.386695,0.87141,-0.819277,-1.046703,0,0.382405
2203,0.470227,-0.191012,-0.021301,0.331509,0.331509,0.509411,1.177575,-0.632400,1.164768,1.189085,...,0.0,-0.421049,-0.421049,0.422054,1.386695,0.87141,-0.819277,-1.046703,0,0.382405


### Creating Failure Flag (failure_flag)
---
Indicator that shows where the system should break down

#### Based on the description of the dataset
---
- Sensor | Failure Threshold
- Cooler Condition (%) | ≤ 3
- Valve Condition (%) | ≤ 80
- Internal Pump Leakage | ≥ 2
- Accumulator Pressure (bar) | ≤ 100

In [ ]:
def create_failure_flag(row):

  failure_flag = ((row["cooler_condition"] <= 3) |
  (row["valve_condition"] <= 70) |
  (row["internal_pump_leakage"] >= 2) |
  (row["hydraulic_accumulator"] <= 90)) and row["stable_flag"]

  return int(failure_flag)

In [ ]:
stat_freq_data["failure_flag"] = stat_freq_data.apply(lambda row: create_failure_flag(row), axis = 1)
stat_freq_data

,meanFS1,stdFS1,minFS1,maxFS1,rangeFS1,rmsFS1,meanFS2,stdFS2,minFS2,maxFS2,...,CP_spectral_flatness,SE_spectral_centroid,SE_spectral_entropy,SE_spectral_flatness,valve_condition,internal_pump_leakage,hydraulic_accumulator,stable_flag,maintenance_score,failure_flag
0,0.495101,-0.130443,-0.021301,-3.143240,-3.143235,0.546497,1.458639,18.562454,1.206381,1.677334,...,0.186150,0.071530,3.126806,0.300456,0.87141,-0.819277,1.387554,1,0.360636,1
1,0.500427,-0.179022,-0.021301,-3.138813,-3.138809,0.548169,1.677960,0.497722,1.653167,1.693157,...,0.100353,0.070487,3.108935,0.320467,0.87141,-0.819277,1.387554,1,0.360636,1
2,0.503533,-0.136685,-0.021301,-3.169798,-3.169794,0.556338,1.595919,2.288859,1.530520,1.670553,...,0.042704,0.070951,3.118370,0.298354,0.87141,-0.819277,1.387554,1,0.360636,1
3,0.505511,-0.105318,-0.021301,-2.731581,-2.731577,0.561995,1.454379,2.385506,1.410063,1.532668,...,0.046708,0.071274,3.123157,0.311515,0.87141,-0.819277,1.387554,1,0.360636,1
4,0.476211,-0.150963,-0.021301,-2.775846,-2.775842,0.520934,1.309819,2.987783,1.274275,1.374439,...,0.036692,0.069130,3.085876,0.309750,0.87141,-0.819277,1.387554,1,0.360636,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,0.475845,-0.204087,-0.021301,0.216421,0.216422,0.515060,1.191293,-0.860944,1.204191,1.175523,...,0.021002,0.067834,3.056402,0.310444,0.87141,-0.819277,-1.046703,0,0.382405,0
2201,0.478025,-0.181232,-0.021301,0.393479,0.393479,0.520099,1.176268,-0.620172,1.173529,1.195866,...,0.018977,0.067764,3.053143,0.306332,0.87141,-0.819277,-1.046703,0,0.382405,0
2202,0.479086,-0.257953,-0.021301,-0.131053,-0.131052,0.513611,1.172717,-0.600236,1.160388,1.177783,...,0.022524,0.068579,3.068287,0.283098,0.87141,-0.819277,-1.046703,0,0.382405,0
2203,0.470227,-0.191012,-0.021301,0.331509,0.331509,0.509411,1.177575,-0.632400,1.164768,1.189085,...,0.015761,0.067792,3.053789,0.276487,0.87141,-0.819277,-1.046703,0,0.382405,0


###Creating Remaining Useful Life (RUL)

In [ ]:
# RUL = Time To Failure and we calculating it based on the failure_flag
def compute_ttf(failure_flags):
    ttf = np.zeros_like(failure_flags, dtype=int)
    next_failure = None
    for i in reversed(range(len(failure_flags))):
        if failure_flags[i]:
            next_failure = i
        ttf[i] = (next_failure - i) if next_failure is not None else len(failure_flags)
    return ttf

In [ ]:
stat_freq_data["rul"] = compute_ttf(stat_freq_data["failure_flag"].values)

### Creating (multi-class) Health State Classification

- Health State Classes
  - Class 0 : Healthy
  - Class 1 : Degraded
  - Class 2 : Failing

In [ ]:
# Health State Classification (multi-class)
def compute_health_state(rul_series):
    values = [0, 1, 2]  # 0: Healthy, 1: Degraded, 2: Failing
    return pd.cut(rul_series, bins=[-1, 10, 30, float('inf')], labels=[2,1,0], right=True).astype(int)

In [ ]:
stat_freq_data["health_state"] = compute_health_state(stat_freq_data["rul"])
stat_freq_data.health_state.value_counts()

,count
health_state,
2,1106
1,700
0,399


### Creating Mode Classification (multi-label flag)


Explanation

In [ ]:
# Failure Mode Classification (multi-label flags)
def compute_failure_modes(df):
    failure_modes = pd.DataFrame()
    failure_modes["cooler_failure"] = (df["cooler_condition"] <= 20).astype(int)
    failure_modes["valve_failure"] = (df["valve_condition"] <= 80).astype(int)
    failure_modes["pump_failure"] = (df["internal_pump_leakage"] >= 1).astype(int)
    failure_modes["hydraulic_failure"] = (df["hydraulic_accumulator"] <= 100).astype(int)
    return failure_modes

In [ ]:
failure_modes_df = compute_failure_modes(stat_freq_data)
stat_freq_data = pd.concat([stat_freq_data, failure_modes_df], axis=1)
stat_freq_data

,meanFS1,stdFS1,minFS1,maxFS1,rangeFS1,rmsFS1,meanFS2,stdFS2,minFS2,maxFS2,...,hydraulic_accumulator,stable_flag,maintenance_score,failure_flag,rul,health_state,cooler_failure,valve_failure,pump_failure,hydraulic_failure
0,0.495101,-0.130443,-0.021301,-3.143240,-3.143235,0.546497,1.458639,18.562454,1.206381,1.677334,...,1.387554,1,0.360636,1,0,2,1,1,0,1
1,0.500427,-0.179022,-0.021301,-3.138813,-3.138809,0.548169,1.677960,0.497722,1.653167,1.693157,...,1.387554,1,0.360636,1,0,2,1,1,0,1
2,0.503533,-0.136685,-0.021301,-3.169798,-3.169794,0.556338,1.595919,2.288859,1.530520,1.670553,...,1.387554,1,0.360636,1,0,2,1,1,0,1
3,0.505511,-0.105318,-0.021301,-2.731581,-2.731577,0.561995,1.454379,2.385506,1.410063,1.532668,...,1.387554,1,0.360636,1,0,2,1,1,0,1
4,0.476211,-0.150963,-0.021301,-2.775846,-2.775842,0.520934,1.309819,2.987783,1.274275,1.374439,...,1.387554,1,0.360636,1,0,2,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,0.475845,-0.204087,-0.021301,0.216421,0.216422,0.515060,1.191293,-0.860944,1.204191,1.175523,...,-1.046703,0,0.382405,0,2205,0,1,1,0,1
2201,0.478025,-0.181232,-0.021301,0.393479,0.393479,0.520099,1.176268,-0.620172,1.173529,1.195866,...,-1.046703,0,0.382405,0,2205,0,1,1,0,1
2202,0.479086,-0.257953,-0.021301,-0.131053,-0.131052,0.513611,1.172717,-0.600236,1.160388,1.177783,...,-1.046703,0,0.382405,0,2205,0,1,1,0,1
2203,0.470227,-0.191012,-0.021301,0.331509,0.331509,0.509411,1.177575,-0.632400,1.164768,1.189085,...,-1.046703,0,0.382405,0,2205,0,1,1,0,1


# Model training

**NOTE:** needs change to switch from classifiers to regressors and ultimately predict maintenance_score.

### splitting

In [ ]:
from sklearn.model_selection import train_test_split

X = total_data.iloc[:, :-1]
y = total_data.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### fitting and testing models

Function to get the score of each model

In [ ]:
def get_score(model, X_train, X_test ,y_train, y_test):
  model.fit(X_train, y_train)
  return model.score(X_test, y_test)

In [ ]:
# importing the  models
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
import xgboost as xgb

In [ ]:
print("rf score:" + str(get_score(RandomForestRegressor(), X_train, X_test, y_train, y_test)))
print("svm score:" + str(get_score(SVC(), X_train, X_test, y_train, y_test)))
print("xgb score:" + str(get_score(xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42), X_train, X_test, y_train, y_test)))

rf score:0.969760184216139


I'm going to try to use kfold cross validation

In [ ]:
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)

scores_svm = [] #list with the svm scores
scores_rf = [] #list with the random forest scores

for train_index, test_index in folds.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    scores_svm.append(get_score(SVC(gamma='auto'), X_train, X_test, y_train, y_test))
    scores_rf.append(get_score(RandomForestClassifier(n_estimators=40), X_train, X_test, y_train, y_test))


ValueError: Supported target types are: ('binary', 'multiclass'). Got 'continuous' instead.

In [ ]:
scores_svm

In [ ]:
scores_rf

function that calculates the average score of a model

In [ ]:
def get_av(score_list):
  av_score = np.median(score_list)
  return float(av_score) #Thanos: float to return actual float not np.float

In [ ]:
get_av(scores_svm)

In [ ]:
get_av(scores_rf)

we can see that the svm model is more acurate than the random forest model and the xgb tree model

### Create Sensor Drift